In [1]:
import sklearn
import time
import pandas as pd   
from matplotlib import pyplot
from numpy import mean
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB, CategoricalNB
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier



In [2]:
def robust_scale(df):
    numerical_columns = df.select_dtypes(include=['number']).columns
    scaler = RobustScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

    return df

def knn_impute_numerical_columns(df, n_neighbors=100):
    df_imputed = df.copy()
    df_imputed.drop(columns=['RecordID','hospital_id'], inplace=True)
    # df_imputed = pd.get_dummies(drop_first=True, data=df_imputed)
    
    categorical_columns = df_imputed.select_dtypes(exclude=['number']).columns
    numerical_cols = df_imputed.select_dtypes(include=['number']).columns
    
    imputer = KNNImputer(n_neighbors=n_neighbors)
    # imputer = SimpleImputer(strategy='mean') 
    df_imputed[numerical_cols] = imputer.fit_transform(df_imputed[numerical_cols])
    
    # Label encode categorical columns
    # label_encoders = {}
    # for col in categorical_columns:
    #     label_encoders[col] = LabelEncoder()
    #     df_imputed[col] = label_encoders[col].fit_transform(df_imputed[col])

    # imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
    # imputer = SimpleImputer(strategy='most_frequent')
    # df_imputed[categorical_columns] = imputer.fit_transform(df_imputed[categorical_columns])
    
    # cat_imputer = SimpleImputer(strategy='most_frequent')
    # df_imputed[categorical_columns] = cat_imputer.fit_transform(df_imputed[categorical_columns])

    return df_imputed


In [3]:
df = pd.read_csv('train.csv')



In [4]:
robust_scale(df)

,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis,hospital_death
0,-1.00000,-0.666667,1.241270,Caucasian,M,Floor,transfer,Med-Surg ICU,Metabolic,Metabolic,...,-0.805556,-1.000000,-0.805556,NaN,NaN,-0.333333,-0.333333,0.0,0.0,0.0
1,-0.99996,-0.789474,0.012698,African American,M,Accident & Emergency,admit,Med-Surg ICU,Cardiovascular,Cardiovascular,...,-0.555556,-0.945946,-0.555556,2.164706,-0.500,NaN,NaN,0.0,0.0,0.0
2,-0.99992,-0.429825,-0.073016,Caucasian,M,Floor,admit,MICU,Respiratory,Respiratory,...,1.305556,1.189189,1.305556,0.129412,0.125,2.750000,2.166667,0.0,0.0,0.0
3,-0.99988,-0.815789,0.860317,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Neurological,Neurologic,...,0.694444,0.891892,0.694444,1.282353,-0.375,0.583333,0.666667,0.0,0.0,0.0
4,-0.99984,0.745614,0.968254,Caucasian,F,Accident & Emergency,admit,CSICU,Cardiovascular,Cardiovascular,...,0.111111,-0.297297,0.111111,-0.458824,-0.250,0.833333,0.833333,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.99984,-0.807018,0.473016,Caucasian,M,Accident & Emergency,admit,Med-Surg ICU,Musculoskeletal/Skin,Undefined Diagnoses,...,-0.472222,-0.864865,-0.472222,-0.600000,-0.250,-0.250000,-0.166667,0.0,0.0,0.0
49996,0.99988,-0.412281,0.568254,Caucasian,F,Floor,admit,Med-Surg ICU,Neurological,Neurologic,...,0.833333,0.405405,0.833333,-0.047059,-0.375,-0.250000,-0.333333,0.0,0.0,0.0
49997,0.99992,-0.956140,1.180952,Caucasian,F,Accident & Emergency,transfer,Med-Surg ICU,Sepsis,Cardiovascular,...,-0.833333,-0.405405,-0.833333,-0.376471,NaN,0.333333,0.500000,1.0,0.0,0.0
49998,0.99996,0.245614,0.571429,Caucasian,M,Operating Room / Recovery,admit,Cardiac ICU,Cardiovascular,Cardiovascular,...,0.611111,0.189189,0.611111,-0.352941,-0.125,-0.416667,-0.333333,0.0,0.0,0.0


In [5]:
df_imputed = knn_impute_numerical_columns(df, n_neighbors=100)

In [6]:
df_dropped = df_imputed.copy().dropna(axis=0)
df_dropped.shape
# missing_value_counts(df_dropped)

(49103, 56)

In [7]:
df_onehot = pd.get_dummies(df_dropped)
df_onehot
df_onehot.shape

(49103, 94)

In [8]:
def label_encode_categorical_columns(df):
    df_encoded = df.copy()
    label_encoder = LabelEncoder()
    categorical_cols = df_encoded.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        df_encoded[col] = label_encoder.fit_transform(df_encoded[col])
    return df_encoded

# Example usage:
# df_dropped = your_dataframe_here
# df_encoded = label_encode_categorical_columns(df_dropped)
# print(df_encoded)


In [72]:
df_onehot = label_encode_categorical_columns(df_onehot)

In [73]:
df_onehot

,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,age,elective_surgery,...,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis,hospital_death
0,1.241270,2,1,1,2,5,5,3,-1.608696,0.0,...,-0.805556,-1.000000,-0.805556,0.040000,0.0300,-0.333333,-0.333333,0.0,0.0,0.0
1,0.012698,0,1,0,0,5,0,0,0.173913,0.0,...,-0.555556,-0.945946,-0.555556,2.164706,-0.5000,0.018333,0.060000,0.0,0.0,0.0
2,-0.073016,2,1,1,0,4,8,6,0.826087,0.0,...,1.305556,1.189189,1.305556,0.129412,0.1250,2.750000,2.166667,0.0,0.0,0.0
3,0.860317,2,1,0,0,5,7,4,-0.260870,0.0,...,0.694444,0.891892,0.694444,1.282353,-0.3750,0.583333,0.666667,0.0,0.0,0.0
4,0.968254,2,0,0,0,1,0,0,0.869565,0.0,...,0.111111,-0.297297,0.111111,-0.458824,-0.2500,0.833333,0.833333,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.473016,2,1,0,0,5,6,8,-1.000000,0.0,...,-0.472222,-0.864865,-0.472222,-0.600000,-0.2500,-0.250000,-0.166667,0.0,0.0,0.0
49996,0.568254,2,0,1,0,5,7,4,-0.173913,0.0,...,0.833333,0.405405,0.833333,-0.047059,-0.3750,-0.250000,-0.333333,0.0,0.0,0.0
49997,1.180952,2,0,0,2,5,9,0,-0.652174,0.0,...,-0.833333,-0.405405,-0.833333,-0.376471,0.0675,0.333333,0.500000,1.0,0.0,0.0
49998,0.571429,2,1,2,0,3,0,0,-0.478261,1.0,...,0.611111,0.189189,0.611111,-0.352941,-0.1250,-0.416667,-0.333333,0.0,0.0,0.0


In [8]:
X = df_onehot.loc[:, df_onehot.columns != "hospital_death"]
y = df_onehot[["hospital_death"]]

X.shape

(49103, 93)

In [9]:
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier(n_estimators=1000, max_depth=7, random_state=0, n_jobs=3)
estimator.fit(X, y)

# Get feature importances
feature_importances = estimator.feature_importances_


feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

feature_num = 15
selected_features = feature_importance_df['Feature'][:feature_num]
selected_features

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


44    apache_4a_hospital_death_prob
45         apache_4a_icu_death_prob
8                  gcs_motor_apache
22                      d1_spo2_min
5               apache_3j_diagnosis
7                   gcs_eyes_apache
24         d1_sysbp_noninvasive_min
23                     d1_sysbp_min
15                ventilated_apache
14                      temp_apache
10                gcs_verbal_apache
25                      d1_temp_min
19                       d1_mbp_min
20           d1_mbp_noninvasive_min
16                    d1_diasbp_min
Name: Feature, dtype: object

In [10]:
X = X[selected_features]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [12]:
def roc_auc_cv(model, X, y):
    cv = RepeatedKFold(n_splits=10, n_repeats=2, random_state=1)
    predicted_probabilities = cross_val_score(model, X, y, scoring="roc_auc", cv=cv, n_jobs=-1, verbose=2)

    return mean(predicted_probabilities)
def calculate_roc_auc(model, X_test, y_test):
    md_probs = model.predict_proba(X_test)
    md_probs = md_probs[:, 1]
    md_auc = roc_auc_score(y_test, md_probs)
    return md_auc

# roc_auc = calculate_roc_auc(model, X_test, y_test)
# print(roc_auc)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, roc_auc_score
import numpy as np

# Define the parameter grid you want to search
param_grid = {
    'n_estimators': [1000],
    'max_depth': [8],
    'min_samples_leaf': [100, 200, 300],
    'min_samples_split': [100, 200, 300],
    # 'max_features': [0.7, 0.8, 0.9],
    # 'criterion': ['gini', 'entropy'],
    # 'bootstrap': [True, False]
}

# Create the RandomForestClassifier
model = RandomForestClassifier(random_state=0, n_jobs=3)

# Create a function to calculate ROC AUC with cross-validation
def calculate_roc_auc(model, X, y):
    y_pred_proba = model.predict_proba(X)[:, 1]
    return roc_auc_score(y, y_pred_proba)

roc_auc_scorer = make_scorer(calculate_roc_auc)

# Create the GridSearchCV object
grid_search = GridSearchCV(model, param_grid, scoring=roc_auc_scorer, cv=5, n_jobs=3)

# Fit the grid search to your data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and corresponding ROC AUC score
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best ROC AUC Score: ", grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
test_roc_auc = calculate_roc_auc(best_model, X_test, y_test)
print("ROC AUC on Test Set: ", test_roc_auc)


In [106]:
model = RandomForestClassifier(n_estimators=1200, max_depth=6, random_state=0, n_jobs=3,
                               min_samples_leaf=200, min_samples_split=350, max_features=0.8, criterion='gini', bootstrap=True)

model.fit(X_train, y_train)
print(calculate_roc_auc(model, X_test, y_test))
# roc_auc_cv(model, X, y)


/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.8733072029757758


In [ ]:

base_classifier = DecisionTreeClassifier(max_depth=7, min_samples_leaf=50, min_samples_split=50)
model = AdaBoostClassifier(base_classifier, n_estimators=100, random_state=42)

# model.fit(X_train, y_train)
roc_auc_cv(model, X, y)

In [13]:
base_classifier = KNeighborsClassifier(n_neighbors=300)
model = BaggingClassifier(base_classifier,bootstrap_features=True , n_estimators=900, random_state=42, n_jobs=4, )

model.fit(X_train, y_train)
calculate_roc_auc(model, X_test, y_test)

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=700, min_samples_leaf=252, min_samples_split=140, learning_rate=0.2, max_depth=2, random_state=42, subsample=0.8)
model.fit(X_train, y_train)
calculate_roc_auc(model, X_test, y_test)

/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8831853483567612

In [138]:
model = GradientBoostingClassifier()

# Define the parameter grid you want to search
param_grid = {
    'n_estimators': [500, 600, 400],  # You can adjust these values
    'learning_rate': [0.18, 0.2, 0.3],  # You can adjust these values
    'max_depth': [1, 2],  # You can adjust these values
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=4)

# Fit the grid search to your training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and corresponding accuracy score
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy Score:", grid_search.best_score_)

# Get the best model from the grid search
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_


/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:424: D

In [ ]:
df_catboost = pd.read_csv('train.csv')
df_catboost = pd.get_dummies(df_catboost)
X = df_catboost.loc[:, df_catboost.columns != "hospital_death"]
y = df_catboost[["hospital_death"]]
# X = X[selected_features]


In [ ]:
X.shape 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
from catboost import CatBoostClassifier, Pool


model = CatBoostClassifier(
    iterations=590,
    depth=5,
    learning_rate=0.09,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=69,
    verbose=20,
    l2_leaf_reg=2,
    border_count=80,
    leaf_estimation_iterations=10,
    # subsample=0.8,
    # min_data_in_leaf=3,
    # colsample_bylevel=0.8,
    
)
# roc_auc_cv(model, X, y)
model.fit(X_train, y_train)
calculate_roc_auc(model, X_test, y_test)

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'iterations': [375],
    'depth': [5],
    'learning_rate': [ 0.09 ],
    'l2_leaf_reg': [1, 2, 3],
    'border_count': [60, 70, 50],  
    'leaf_estimation_iterations': [20, 30, 10],
    
}

# Create the CatBoostClassifier
model = CatBoostClassifier(loss_function='Logloss', eval_metric='AUC', random_seed=42, verbose=20)

# Create a GridSearchCV object
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc', n_jobs=3)

# Fit the grid search to your data
grid_search.fit(X, y)

# Print the best hyperparameters
print("Best Parameters: ", grid_search.best_params_)
print("Best AUC Score: ", grid_search.best_score_)


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Define the XGBoost classifier with default parameters
model = XGBClassifier(random_state=42)

# Define the parameter grid for the grid search
param_grid = {
    'n_estimators': [800, 900, 700],
    # 'max_depth': [4, 5],
    'learning_rate': [0.038, 0.039, 0.037],
    # 'min_child_weight': [1, 2],
    'colsample_bytree': [0.8, 0.83, 0.77],
    # 'alpha': [0.1],
    # 'eval_metric': ['logloss'],
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=2)

# Fit the grid search to your data (X, y)
grid_search.fit(X, y)

# Print the best parameters and the corresponding ROC AUC score
print("Best Parameters: ", grid_search.best_params_)
print("Best ROC AUC Score: ", grid_search.best_score_)


In [62]:
X_train

,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,gcs_motor_apache,d1_spo2_min,gcs_eyes_apache,d1_sysbp_noninvasive_min,d1_sysbp_min,ventilated_apache,temp_apache,apache_3j_diagnosis,...,icu_admit_source,elective_surgery,icu_type,apache_post_operative,ethnicity,solid_tumor_with_metastasis,immunosuppression,gcs_unable_apache,gender,icu_stay_type
18165,-0.083333,-0.166667,0.0,-0.166667,0.0,-0.884615,-0.884615,0.0,-2.8000,0.787968,...,0,0.0,5,0.0,2,0.0,0.0,0.0,0,0
36059,0.833333,0.666667,0.0,0.500000,0.0,-0.153846,-0.153846,1.0,-2.4000,-0.401984,...,1,0.0,4,0.0,0,0.0,0.0,0.0,0,0
13242,0.416667,0.500000,0.0,-0.333333,0.0,-0.615385,-0.615385,0.0,0.2156,0.188072,...,0,0.0,5,0.0,0,0.0,0.0,0.0,0,0
32985,4.583333,5.333333,-2.0,-1.333333,-3.0,0.115385,0.115385,1.0,0.6000,-0.389984,...,1,0.0,5,0.0,2,0.0,0.0,0.0,1,2
41133,-0.166667,-0.166667,0.0,0.000000,0.0,1.807692,1.807692,0.0,0.4000,-0.601976,...,0,0.0,5,0.0,2,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43723,0.000000,0.000000,0.0,0.500000,0.0,0.923077,0.923077,0.0,-2.6000,-0.401984,...,0,0.0,7,0.0,2,0.0,0.0,0.0,0,0
32511,-0.326667,-0.963333,0.0,0.166667,0.0,-0.807692,-0.807692,0.0,0.6000,0.594096,...,0,0.0,5,0.0,2,0.0,0.0,0.0,1,0
5192,1.916667,2.500000,-1.0,-1.666667,0.0,0.192308,0.192308,1.0,0.2000,-0.405984,...,1,0.0,4,0.0,3,0.0,0.0,0.0,1,0
12172,2.666667,4.166667,0.0,-4.000000,-1.0,-1.730769,-1.730769,1.0,-0.2000,-0.605976,...,0,0.0,0,0.0,2,0.0,0.0,0.0,0,0


In [ ]:
X = df.loc[:, df.columns != "hospital_death"]
y = df[["hospital_death"]]

X.shape

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=800,  
    max_depth=4,
    learning_rate=0.038,
    random_state=49,
    min_child_weight=1,
    colsample_bytree=0.78,
    alpha= 0.1,
    eval_metric='logloss',
)
# roc_auc_cv(model, X, y)

# model.fit(X_train, y_train)
roc_auc_cv(model, X, y)
# calculate_roc_auc(model, X_test, y_test)
# print(roc)
# model.get_params()

In [ ]:
X_train.shape

In [ ]:
model = MLPClassifier(
    hidden_layer_sizes=(64,32, 16), 
    activation='relu', 
    max_iter=10, 
    random_state=42, 
    solver='adam',  # You can adjust other hyperparameters as needed
    learning_rate_init=0.001,
    shuffle=True,    
)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=300)
model.fit(X_train, y_train)
roc_auc_cv(model, X, y)


In [ ]:
X.shape

In [ ]:
# model = RandomForestClassifier(n_estimators=100, max_depth=17, min_samples_leaf=100, min_samples_split=200, random_state=0, n_jobs=3)
model.fit(X_train, y_train)
roc_auc_cv(model, X, y)
# calculate_roc_auc(model, X_test, y_test)


In [ ]:
# model = RandomForestClassifier(n_estimators=100, max_depth=17, min_samples_leaf=100, min_samples_split=200, random_state=0, n_jobs=3)
model.fit(X_train, y_train)
# roc_auc_cv(model, X, y)
calculate_roc_auc(model, X_test, y_test)


In [ ]:
model = best_model

In [ ]:
df_catboost = pd.read_csv('train.csv')
robust_scale(df_catboost)
df_catboost = knn_impute_numerical_columns(df_catboost, n_neighbors=10)
df_catboost = pd.get_dummies(df_catboost)




In [ ]:

X = df_catboost.loc[:, df_catboost.columns != "hospital_death"]
y = df_catboost[["hospital_death"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train.shape


In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=590,
    depth=5,
    learning_rate=0.09,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=69,
    verbose=20,
    l2_leaf_reg=2,
    border_count=80,
    leaf_estimation_iterations=20,
    # scale_pos_weight=0.4,
    # subsample=0.8,
    # min_data_in_leaf=1,
    # colsample_bylevel=0.8,
    
)
# roc_auc_cv(model, X, y)
model.fit(X_train, y_train)
calculate_roc_auc(model, X_test, y_test)

In [131]:
model.fit(X, y)


/opt/homebrew/Caskroom/miniforge/base/envs/imlenv/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(learning_rate=0.3, max_depth=1, n_estimators=500,
                           random_state=42)

In [132]:
X.shape

(50000, 93)

In [19]:
def generate_predictions_for_model(model, test_file, output_file):
    
    df = pd.read_csv(test_file)
    record_ids = df["RecordID"]
    
    robust_scale(df)
    df = knn_impute_numerical_columns(df, n_neighbors=10)
    # df = df.dropna(axis=0)
    df = pd.get_dummies(df)
    # df = label_encode_categorical_columns(df)
    
    X_test = df[selected_features] 
    
    # X_test = df.loc[:, df.columns != "hospital_death"]
    
    probs = model.predict_proba(X_test)
    probs = probs[:, 1]
    
    # Create a DataFrame for the results
    result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
    # Save the results to a CSV file
    result.to_csv(output_file, index=False, header=["RecordID", "hospital_death"])


In [28]:
generate_predictions_for_model(model, "test.csv", "results82.csv")


In [ ]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

params = {
    'objective': 'binary',  
    'metric': 'binary_logloss', 
    'boosting_type': 'gbdt',  
    'num_leaves': 31,  
    'learning_rate': 0.05,
    'feature_fraction': 0.75,  
    'bagging_fraction': 0.90,  
    'bagging_freq': 5,  # 
    'verbose': 1,
    'max_bin': 255,
    'min_data_in_leaf': 40,  # Example value, adjust as needed
    'min_sum_hessian_in_leaf': 1e-4  ,# Example value, adjust as needed


}

num_round = 211# Number of boosting iterations
lightgbm_model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

y_pred_prob = lightgbm_model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)
# roc_auc_cv(lightgbm_model, X, y)



In [ ]:
params = {
    'objective': 'binary',  # Binary classification
    'metric': 'binary_logloss',  # Logarithmic loss for binary classification
    'boosting_type': 'gbdt',  # Gradient boosting decision tree
    'num_leaves': 31,  # Maximum number of leaves in each tree
    'learning_rate': 0.05,  # Step size shrinkage
    'feature_fraction': 0.75,  # Fraction of features to be used in each iteration
    'bagging_fraction': 0.90,  # Fraction of data to be randomly sampled for training
    'bagging_freq': 5,  # Frequency for bagging
    'verbose': 0  # Verbosity level,    
}

num_round = 210  # Number of boosting iterations
lightgbm_model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

y_pred_prob = lightgbm_model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)






In [ ]:
def gen_resultfile_lbgm(model, test_file, output_file):
    
    df_test = pd.read_csv(test_file)
    record_ids = df_test["RecordID"]
    
    df_scaled = robust_scale(df_test)
    df_test_imputed = knn_impute_numerical_columns(df_scaled, n_neighbors=100)
    df_test_onehot = pd.get_dummies(df_test_imputed)
    
    X_test = df_test_onehot[selected_features] 
    
    # X_test = df_test_onehot.loc[:, df_test_onehot.columns != "hospital_death"]
    
    probs = model.predict(X_test)
    
    # Create a DataFrame for the results
    result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
    # Save the results to a CSV file
    result.to_csv(output_file, index=False, header=["RecordID", "hospital_death"])

In [ ]:
gen_resultfile_lbgm(lightgbm_model, "test.csv", "results54.csv")

In [ ]:
params = {
    'objective': 'binary',  # Binary classification
    'metric': 'binary_logloss',  # Logarithmic loss for binary classification
    'boosting_type': 'gbdt',  # Gradient boosting decision tree
    'num_leaves': 31,  # Maximum number of leaves in each tree
    'learning_rate': 0.05,  # Step size shrinkage
    'feature_fraction': 0.75,  # Fraction of features to be used in each iteration
    'bagging_fraction': 0.90,  # Fraction of data to be randomly sampled for training
    'bagging_freq': 5,  # Frequency for bagging
    'verbose': 0  # Verbosity level,    
}

num_round = 210  # Number of boosting iterations
lightgbm_model = lgb.train(params, train_data, num_round, valid_sets=[test_data])

y_pred_prob = lightgbm_model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)



# Make predictions using the trained model
record_ids = X["RecordID"]
probs = model.predict(X_test)

result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
# Save the results to a CSV file
result.to_csv("results54", index=False, header=["RecordID", "hospital_death"])

In [ ]:
y_pred_prob = lightgbm_model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)


In [ ]:
X.shape

In [ ]:
train_data = pd.read_csv('train.csv')  # Replace with your training dataset path
test_data = pd.read_csv('test.csv')    # Replace with your other dataset path


# Create a LightGBM dataset from the training data
train_dataset = lgb.Dataset(X, label=y)

# Define LightGBM parameters
params = {
    'objective': 'binary',  # Binary classification
    'metric': 'binary_logloss',  # Logarithmic loss for binary classification
    'boosting_type': 'gbdt',  # Gradient boosting decision tree
    'num_leaves': 31,  # Maximum number of leaves in each tree
    'learning_rate': 0.05,  # Step size shrinkage
    'feature_fraction': 0.75,  # Fraction of features to be used in each iteration
    'bagging_fraction': 0.90,  # Fraction of data to be randomly sampled for training
    'bagging_freq': 5,  # Frequency for bagging
    'verbose': 0  # Verbosity level,    
}

num_boost_round = 210  # Number of boosting rounds

# Train the LightGBM model on the training dataset
model = lgb.train(params, train_dataset, num_boost_round=num_boost_round)

# Prepare the other dataset for predictions
X_test = test_data  # Preprocess your other dataset as needed

# Make predictions using the trained model
record_ids = X["RecordID"]
probs = lightgbm_model.predict(X_test)

result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
# Save the results to a CSV file
result.to_csv("results54", index=False, header=["RecordID", "hospital_death"])



In [ ]:
test_df = pd.read_csv("test.csv")
robust_scale_df = robust_scale(test_df)
df_test_imputed = knn_impute_numerical_columns(robust_scale_df, n_neighbors=100)
df_test_onehot = pd.get_dummies(df_test_imputed)
X_test = df_test_onehot[selected_features]
X_test.shape

In [ ]:
y_pred_prob = lightgbm_model.predict(X_test)
y_pred_prob


In [ ]:
record_ids = test_df["RecordID"]
result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
# Save the results to a CSV file
result.to_csv("results54.csv", index=False, header=["RecordID", "hospital_death"])